# Track Pattern Recognition using Hough Transform and Tracks Classification

## Introduction

Track pattern recognition is an early step of the reconstruction of data coming from a particle detector. It recognizes tracks among the subdetectors hits. Reconstructed track parameters allow to estimate the particle deviation in a magnetic field, and thus reconstruct its charge and momentum. This information is used for the reconstruction of the decay vertex, to identify the mother particle and for further particle identification.

There is wide variety of the track pattern recognition methods. They differ in how they process the hits, what kind of tracks they are able to recognize and which requirements these tracks should satisfy. Therefore, specifics of an experiment and the detector geometry affect the tracking performance and track pattern recognition methods should be adapted to it accordingly.

In this notebook a track pattern recognition for a 2D detector with circular geometry and uniform magnetic field is considered. The detector schema with hits and tracks of an event is shown in the figure below. The challenge is to recognize tracks of an event with the highest efficiecny. It supposed that one hit can belong to only one track. 

<img src="pic/detector.png" /> <br>

## About this notebook

This notebook demonstrate how Hough Transform and Machine Learning can be used for track pattern recognition. The notebook describes input data, the track pattern recognition method and qualyti metrics, and shows how to use them.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandas
import numpy

import user_test_submission as submission

# Input data

In [2]:
data = pandas.read_csv('public_train.csv', index_col=False)
#data = data[data['event_id'].values <= 100]

data.head()

,event_id,cluster_id,layer,iphi,x,y
0,3,4,4,53253,53.900430,-265.585662
1,3,1,5,37216,-47.614439,-402.191329
2,3,1,0,7181,-4.253919,-38.767308
3,3,3,2,7937,44.418132,148.499258
4,3,4,0,7657,7.588600,-38.254583


## Data Preparation

In [3]:
X = data[[u'event_id', u'layer', u'iphi', u'x', u'y']].values
y = data[[u'event_id', u'cluster_id']].values

# Split Data into Train/Test Samples

In [4]:
from sklearn.model_selection import train_test_split

event_ids = numpy.unique(data['event_id'].values)

event_ids_train, event_ids_test = train_test_split(event_ids, 
                                                   test_size=0.5, 
                                                   random_state=42)

X_train, y_train = X[data['event_id'].isin(event_ids_train), :], y[data['event_id'].isin(event_ids_train)]
X_test, y_test = X[data['event_id'].isin(event_ids_test), 1:], y[data['event_id'].isin(event_ids_test)]

# Hough Transform

## Track Pattern Recognition

In [40]:
from fast_hough import Clusterer

ctr = Clusterer(invr_size=0.05 / 20000., theta_size=2. * numpy.pi / 450.,  min_hits=2, phi_window=0.5)

ctr.fit(X_train, y_train)

In [41]:
%%time
from metrics import predictor

y_pred_test = predictor(ctr, X_test, y_test)

CPU times: user 3min 15s, sys: 1.15 s, total: 3min 16s
Wall time: 3min 19s


## Score

In [46]:
score = submission.score_function(y_test, y_pred_test)
score

0.9483761622278504

In [30]:
1. *len(y_pred_test[y_pred_test == -1]) / len(y_pred_test)

0.012377635091815546

## Additional quality metrics

In [31]:
from metrics import RecognitionQuality

rq = RecognitionQuality(track_eff_threshold=0.8, min_hits_per_track=4)
report_event, report_tracks = rq.calculate(y_test, y_pred_test)

In [32]:
report_event.mean(axis=0)[1:]

ReconstructionEfficiency    0.906609
GhostRate                   0.075585
CloneRate                   0.001256
AvgTrackEfficiency          0.943791
dtype: float64